In [1]:
stream_dir = '/FileStore/tables/baby-data'
static = spark.read.csv(stream_dir, header=True) 
dataSchema = static.schema 

In [2]:
print(dataSchema)

StructType(List(StructField(state,StringType,true),StructField(sex,StringType,true),StructField(year,StringType,true),StructField(name,StringType,true),StructField(count,StringType,true)))

In [3]:
baby_stream_df = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1).csv(stream_dir)

In [4]:
gender_count = baby_stream_df.groupBy('sex').count()

In [5]:
gender_count_query = gender_count.writeStream.queryName('gender_counts').format('memory').outputMode('complete').start()

In [6]:
from time import sleep
for x in range(5):
  print("Female count number ", x)
  spark.sql("SELECT * FROM gender_counts WHERE sex='F'").show()
  sleep(30)

Female count number 0
+---+-----+
sex|count|
+---+-----+
 F|33417|
+---+-----+

Female count number 1
+---+-----+
sex|count|
+---+-----+
 F|67008|
+---+-----+

Female count number 2
+---+-----+
sex|count|
+---+-----+
 F|67008|
+---+-----+

Female count number 3
+---+------+
sex| count|
+---+------+
 F|167264|
+---+------+

Female count number 4
+---+------+
sex| count|
+---+------+
 F|234184|
+---+------+